# 🚀 Классический пример: IHDP/Actor–Critic для F‑16 (линейная продольная модель)

Лаконичный, удобный для чтения на GitHub пример онлайн‑обучения актор–критик (IHDP/ADP) для продольной динамики F‑16: настройка модели, формирование цели слежения, конфигурация актор/критик, запуск цикла обучения и оценка метрик.

## 📋 Оглавление
- [📖 Введение](#📖-Введение)
- [📦 Импорты](#📦-Импорты)
- [🔧 Параметры эксперимента](#🔧-Параметры-эксперимента)
- [🧠 Модели Actor, Critic и инкрементальная модель](#🧠-Модели-Actor,-Critic-и-инкрементальная-модель)
- [✈️ Инициализация среды F-16](#✈️-Инициализация-среды-F-16)
- [🔄 Тренировочный цикл](#🔄-Тренировочный-цикл)
- [📊 Метрики](#📊-Метрики)
- [📈 Визуализация](#📈-Визуализация)
- [💡 Советы и диагностика](#💡-Советы-и-диагностика)
- [📄 Лицензия](#📄-Лицензия)

## 📖 Введение
Демонстрация IHDP/Actor–Critic для задачи слежения по углу атаки (alpha) в линейной продольной модели F‑16. Пример ориентирован на наглядность и повторяемость результатов.

### 🧠 Что такое IHDP/Actor-Critic?
**IHDP (Incremental Heuristic Dynamic Programming)** — это метод адаптивного управления, основанный на принципах динамического программирования. Алгоритм состоит из трех ключевых компонентов:

1. **Actor (Актор)** — нейронная сеть, которая генерирует управляющие воздействия на основе текущего состояния системы и желаемого референс-сигнала
2. **Critic (Критик)** — нейронная сеть, которая оценивает качество управления через функцию стоимости (value function)
3. **Incremental Model (Инкрементальная модель)** — модель, которая идентифицирует локальную динамику системы в режиме реального времени

### ✈️ Задача управления F-16
В данном примере мы решаем задачу слежения по углу атаки (α) для линейной продольной модели истребителя F-16. Система управления должна обеспечить точное следование заданному референс-сигналу при соблюдении физических ограничений.

**Основные шаги алгоритма:**
- Настройка временной шкалы и референс-сигнала (ступенчатый сигнал 5°);
- Выбор наблюдаемых состояний (α, q) и управляющего канала (стабилизатор);
- Инициализация и обучение Actor, Critic и IncrementalModel;
- Запуск онлайн‑обучения в замкнутом контуре на линейной модели F‑16;
- Анализ метрик качества управления и визуализация результатов.

## 📦 Импорты

Загружаем необходимые библиотеки для реализации алгоритма IHDP/Actor-Critic:

- **tqdm** — для отображения прогресс-бара во время обучения
- **numpy** — для численных вычислений и работы с массивами
- **Actor, Critic, IncrementalModel** — основные компоненты алгоритма IHDP
- **LongitudinalF16** — линейная продольная модель истребителя F-16
- **sinusoid** — генератор синусоидальных сигналов для референс-траекторий

In [ ]:
from tqdm import tqdm
import numpy as np

from tensoraerospace.agent.ihdp.Actor import Actor
from tensoraerospace.agent.ihdp.Critic import Critic
from tensoraerospace.agent.ihdp.Incremental_model import IncrementalModel

from tensoraerospace.aerospacemodel.f16.linear.longitudinal import (
    set_initial_state, initial_state, LongitudinalF16
)
from tensoraerospace.signals import sinusoid

## 🔧 Параметры эксперимента

Определяем ключевые параметры для проведения эксперимента по управлению F-16:

### ⏰ Временные параметры
- **t0, tn, dt** — временные границы и шаг дискретизации для численного интегрирования
- **number_time_steps** — общее количество временных шагов моделирования
- **time** — массив временных отметок для построения графиков

### 📊 Референс-сигнал
Задаем желаемую траекторию для угла атаки. В данном примере используется ступенчатый сигнал 5° на 1000-м шаге (t=10с), что имитирует команду пилота на изменение угла атаки.

### 🎯 Состав векторов состояния и управления
- **selected_states** — наблюдаемые переменные состояния (α - угол атаки, q - угловая скорость тангажа)
- **selected_input** — управляющие воздействия (stab - отклонение стабилизатора)
- **tracking_states** — переменные, по которым осуществляется слежение (только α)
- **input_magnitude_limits, input_rate_limits** — физические ограничения на управление

In [ ]:
# Временные параметры
t0 = 0.0     # начальное время, с
tn = 20.0    # время моделирования, с
dt = 0.01    # шаг дискретизации, с
number_time_steps = int(((tn - t0) / dt) + 1)

# Шкала времени
time = list(np.arange(0, number_time_steps * dt, dt))
# t используется в unit_step как индексная шкала (для шага на 1000-м отсчете)
t = np.linspace(0, len(time), len(time))

# Референс-сигнал: используйте один из вариантов ниже
# reference_signals = np.reshape([np.deg2rad(1) for _ in time], [1, -1])
# reference_signals = np.reshape(np.deg2rad(sinusoid(time, 5, 5)), [1, -1])

def unit_step(x):
    return np.deg2rad(5) * (x > 1000)

reference_signals = np.reshape(unit_step(t), [1, -1])

In [ ]:
# Состав вектора состояния/управления и ограничения
selected_states = ['alpha', 'q']
selected_input = ['stab']
input_magnitude_limits = 25   # предельная величина управляющего воздействия, град
input_rate_limits = 60        # предельная скорость изменения, град/с
tracking_states = ['alpha']

# Индексы слежения для быстрого доступа
indices_tracking_states = [selected_states.index(s) for s in tracking_states]

## 🧠 Модели Actor, Critic и инкрементальная модель

Настраиваем три ключевых компонента алгоритма IHDP с подробным объяснением параметров:

### 🎭 Параметры Actor (Актор)
- **actor_start_training** — шаг, с которого начинается обучение актора (5 шагов задержки для стабилизации)
- **actor_layers** — архитектура нейронной сети (25 нейронов в скрытом слое, 1 выход)
- **actor_activations** — функции активации ('tanh' для нелинейности)
- **actor_learning_rate** — скорость обучения с адаптивным затуханием
- **type_PE** — тип сигнала возбуждения для исследования пространства состояний
- **amplitude_3211, pulse_length_3211** — параметры 3-2-1-1 сигнала возбуждения
- **maximum_input, maximum_q_rate** — физические ограничения на управление

### 🎯 Параметры Critic (Критик)
- **Q_weights** — веса в функции стоимости для штрафа за отклонение от референса
- **gamma** — коэффициент дисконтирования для будущих наград (0.99)
- **critic_learning_rate** — скорость обучения критика
- **critic_layers** — архитектура сети критика (25 нейронов, линейный выход)
- **critic_activations** — активации ('tanh' + 'linear' для оценки стоимости)

### ⚡ Инкрементальная модель
Идентифицирует локальную динамику системы в режиме реального времени, используя метод наименьших квадратов для оценки матрицы Якобиана системы.

In [ ]:
# Параметры актора
actor_start_training = 5
actor_layers = (25, 1)
actor_activations = ('tanh', 'tanh')
actor_learning_rate = 2
actor_learning_rate_cascaded = 1.2
actor_learning_rate_exponent_limit = 10

# Возбуждение (PE)
type_PE = 'combined'
amplitude_3211 = 15
pulse_length_3211 = 5 / dt

# Ограничения
maximum_input = 25
maximum_q_rate = 20

# Прочее
cascaded_actor = False
WB_limits = 30
NN_initial = 130

In [ ]:
# Параметры критика
Q_weights = [8]
tracking_states = ['alpha']
critic_start_training = -1
gamma = 0.99
critic_learning_rate = 15
critic_learning_rate_exponent_limit = 10
critic_layers = (25, 1)
critic_activations = ('tanh', 'linear')

In [ ]:
# Инициализация и сборка моделей
actor = Actor(selected_input, selected_states, tracking_states, indices_tracking_states,
              number_time_steps, actor_start_training, actor_layers, actor_activations,
              actor_learning_rate, actor_learning_rate_cascaded, actor_learning_rate_exponent_limit,
              type_PE, amplitude_3211, pulse_length_3211, WB_limits,
              maximum_input, maximum_q_rate, cascaded_actor, NN_initial)
actor.build_actor_model()

critic = Critic(Q_weights, selected_states, tracking_states, indices_tracking_states,
               number_time_steps, critic_start_training, gamma,
               critic_learning_rate, critic_learning_rate_exponent_limit,
               critic_layers, critic_activations, WB_limits, NN_initial)
critic.build_critic_model()

incremental_model = IncrementalModel(selected_states, selected_input, number_time_steps, dt,
                                     input_magnitude_limits, input_rate_limits)

## ✈️ Инициализация среды F-16

Настраиваем линейную продольную модель истребителя F-16 для проведения эксперимента:

### 🎯 Начальные условия
- **alpha_states** — массив для сохранения истории угла атаки во время моделирования
- **initial_state** — начальное состояние системы с углом атаки 3° (типичное крейсерское значение)
- **model** — экземпляр линейной продольной модели F-16 с выбранными выходными переменными

### 🛩️ Модель LongitudinalF16
Линейная продольная модель F-16 описывает движение самолета в вертикальной плоскости и включает:
- **Переменные состояния**: α (угол атаки), q (угловая скорость тангажа), θ (угол тангажа), h (высота)
- **Управляющие воздействия**: δe (отклонение стабилизатора), δt (положение РУД)
- **Линеаризация**: модель линеаризована относительно установившегося полета на заданной высоте и скорости

В данном примере мы наблюдаем только α и q, управляем только стабилизатором (δe), что упрощает задачу управления до двумерной системы.

In [ ]:
# Начальные условия и создание модели F-16 (линейная продольная)
alpha_states = []
initial_state = set_initial_state({'alpha': np.deg2rad(3)})
model = LongitudinalF16(initial_state, number_time_steps, selected_state_output=['alpha', 'q'])

## 🔄 Тренировочный цикл

Основной цикл онлайн-обучения IHDP/Actor-Critic, выполняющийся на каждом временном шаге:

### 📋 Алгоритм на каждом шаге:
1. **Генерация управления** — Actor вычисляет управляющее воздействие ut на основе текущего состояния xt и референса xt_ref
2. **Идентификация модели** — IncrementalModel обновляет локальную линейную модель системы методом наименьших квадратов
3. **Прогноз состояния** — Предсказание следующего состояния xt+1 на основе идентифицированной модели
4. **Обучение Critic** — Обновление весов критика для улучшения оценки функции стоимости
5. **Обучение Actor** — Корректировка политики актора на основе градиента функции стоимости
6. **Обновление атрибутов** — Сохранение истории для следующей итерации
7. **Шаг модели** — Выполнение одного шага интегрирования модели F-16 с вычисленным управлением

### ⭐ Ключевые особенности:
- **Онлайн-обучение**: веса нейронных сетей обновляются в режиме реального времени
- **Адаптивные скорости обучения**: используется экспоненциальное затухание для стабилизации
- **Физические ограничения**: управление ограничено реалистичными пределами стабилизатора
- **Возбуждающий сигнал**: добавляется для обеспечения исследования пространства состояний

In [ ]:
# Основной цикл онлайн-обучения
xt = np.array([[np.deg2rad(3)], [0]])
alpha_series = []
alpha_ref_series = []
u_series = []

for time_step in tqdm(range(number_time_steps - 1)):
    xt_ref = np.reshape(reference_signals[:, time_step], [-1, 1])

    ut = actor.run_actor_online(xt, xt_ref)
    u_series.append(float(ut))

    # Идентификация инкрементальной модели и прогноз
    G = incremental_model.identify_incremental_model_LS(xt, ut)
    xt1_est = incremental_model.evaluate_incremental_model()

    # Обучение критика и актора
    xt_ref1 = np.reshape(reference_signals[:, time_step + 1], [-1, 1])
    _ = critic.run_train_critic_online_alpha_decay(xt, xt_ref)
    Jt1, dJt1_dxt1 = critic.evaluate_critic(np.reshape(xt1_est, [-1, 1]), xt_ref1)
    actor.train_actor_online_alpha_decay(Jt1, dJt1_dxt1, G, incremental_model, critic, xt_ref1)

    # Обновление атрибутов
    incremental_model.update_incremental_model_attributes()
    critic.update_critic_attributes()
    actor.update_actor_attributes()

    # Шаг модели
    xt = model.run_step(ut)
    alpha_series.append(float(model.get_state('alpha')[-1]))
    alpha_ref_series.append(float(xt_ref[0, 0]))

## 📊 Метрики

Оценка качества управления с помощью ключевых показателей эффективности:

### 📏 Статическая ошибка (Static Error)
Разность между заданным и достигнутым значением угла атаки в установившемся режиме. Измеряется в градусах и показывает точность системы управления в стационарном состоянии. Идеальное значение: 0°.

### 📈 Перерегулирование (Overshoot)
Максимальное превышение заданного значения при переходном процессе, выраженное в процентах от установившегося значения. Характеризует стабильность системы:
- **< 5%** — отличное качество управления
- **5-15%** — приемлемое качество
- **> 15%** — требуется настройка параметров

### 🔍 Диапазон анализа
Перерегулирование вычисляется в диапазоне шагов 1000-15000, что соответствует периоду после подачи ступенчатого сигнала до установления переходного процесса.

In [ ]:
# Статическая ошибка и перерегулирование по углу атаки (в градусах)
static_error_deg = np.rad2deg(reference_signals[0][-1] - model.get_state('alpha')[-1])
overshoot_pct = (
    (np.max(model.get_state('alpha')[1000:15000]) - reference_signals[0][1001])
    * 100 / reference_signals[0][1001]
)
print(f"Статическая ошибка: {static_error_deg} град.")
print(f"Перерегулирование: {overshoot_pct} %")

## 📈 Визуализация

Построение графиков для анализа качества управления и поведения системы:

### 📊 График слежения по углу атаки
Основной результат эксперимента — сравнение фактической траектории угла атаки (сплошная линия) с заданным референс-сигналом (пунктирная линия).

**Что анализировать на графике:**
- **Время переходного процесса** — как быстро система достигает заданного значения после ступенчатого сигнала на 1000-м шаге
- **Перерегулирование** — максимальное превышение целевого значения (5°) в переходном процессе
- **Установившуюся ошибку** — разность между заданным и фактическим значением в конце моделирования
- **Колебательность** — наличие осцилляций вокруг заданного значения

### 🔍 Интерпретация результатов
- **Хорошее качество**: плавный переходный процесс без значительного перерегулирования, быстрое достижение заданного значения
- **Требует улучшения**: большие колебания, медленная сходимость, значительная установившаяся ошибка

Дополнительно можно построить графики управляющего сигнала `u_series` и функции стоимости для более детального анализа.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 4))
plt.plot(np.rad2deg(alpha_series), label='alpha (deg)')
plt.plot(np.rad2deg(alpha_ref_series), '--', label='reference (deg)')
plt.xlabel('Шаг')
plt.ylabel('Градусы')
plt.title('Слежение по углу атаки (alpha)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()

## 💡 Советы и диагностика

Рекомендации по настройке и отладке алгоритма IHDP/Actor-Critic:

### 📊 Настройка референс-сигналов
- **Ступенчатый сигнал**: `unit_step(t)` — для анализа переходных процессов и времени установления
- **Синусоидальный сигнал**: `sinusoid(time, amplitude, frequency)` — для проверки слежения за периодическими командами
- **Постоянный сигнал**: `np.deg2rad(constant_value)` — для анализа точности в установившемся режиме

### ⚖️ Настройка весов функции стоимости
- **Q_weights**: увеличивайте для более агрессивного слежения, уменьшайте для более плавного управления
- **tracking_states**: можно добавить 'q' для одновременного управления углом атаки и угловой скоростью
- **gamma**: коэффициент дисконтирования (0.95-0.99) влияет на горизонт планирования

### 🧠 Архитектура нейронных сетей
- **Размер слоев**: начинайте с 25 нейронов, увеличивайте при сложных задачах (50-100)
- **Функции активации**: 'tanh' для скрытых слоев, 'linear' для выходного слоя критика
- **Скорости обучения**: уменьшайте при нестабильности, увеличивайте при медленной сходимости

### ⚙️ Физические ограничения
- **maximum_input**: предельное отклонение стабилизатора (обычно ±25°)
- **maximum_q_rate**: ограничение угловой скорости для предотвращения резких маневров
- **input_rate_limits**: скорость изменения управления для реалистичности

### 🔧 Диагностика проблем
- **Нестабильность**: уменьшите скорости обучения, проверьте ограничения
- **Медленная сходимость**: увеличьте веса Q_weights, проверьте возбуждающий сигнал
- **Большое перерегулирование**: уменьшите агрессивность актора, настройте критика
- **Воспроизводимость**: зафиксируйте numpy.random.seed() в начале эксперимента

### 📋 Дополнительная диагностика
Для глубокого анализа сохраняйте и визуализируйте:
- Историю управляющих сигналов `u_series`
- Траектории состояний `xt` (угол атаки и угловая скорость)
- Значения функции стоимости `Jt` для контроля обучения критика
- Веса нейронных сетей для анализа сходимости обучения

## 📄 Лицензия
Этот пример распространяется на условиях лицензии проекта (см. файл LICENSE в корне репозитория).